### Exploring Rotterdam Weather
(this tutorial is adapated from [Vega-Altair documentation](https://altair-viz.github.io/case_studies/exploring-weather.html))

In this tutorial, you'll learn a few more techniques for ceating visualizatinos in Altair.

For this tutorial, we will create visualizations to explore weather data for Rotterdam, taken from KNMI. The dataset is a CSV file with columns for the temperature (in Celsius), precipitation (in millimeters), wind speed (in meter/second), and weather type. We have one row for each day from January 1st, 2019 to December 31st, 2022.s:

In [1]:
import pandas as pd
import altair as alt

df = pd.read_csv('https://raw.githubusercontent.com/mattijn/datasets/master/rotterdam_weather_19-22.csv')
df.head()

,date,precipitation,temp_max,temp_min,wind,weather
0,2019-01-01,0.9,9.0,6.0,6.250000,rain
1,2019-01-02,0.4,6.8,2.8,4.333333,rain
2,2019-01-03,2.0,7.0,0.9,1.708333,rain
3,2019-01-04,0.2,6.6,3.5,3.541667,rain
4,2019-01-05,0.0,7.5,5.8,5.125000,rain


The data is loaded from the web and stored in a pandas DataFrame, and from here we can explore it with Altair.

Let’s start by looking at the precipitation, using tick marks to see the distribution of precipitation values:

In [2]:
import altair as alt

alt.Chart(df).mark_tick().encode(
    x='precipitation',
)

alt.Chart(...)

It looks as though precipitation is skewed towards lower values; that is, when it rains in Rotterdam, it usually doesn’t rain very much. It is difficult to see patterns across continuous variables, and so to better see this, we can create a histogram of the precipitation data. For this we first discretize the precipitation values by adding a binning to `x`. Additionally, we set our encoding channel `y` with `count`. The result is a histogram of precipitation values:

In [3]:
alt.Chart(df).mark_bar().encode(
    alt.X('precipitation').bin(),
    y='count()'
)

alt.Chart(...)

Next, let’s look at how precipitation in Rotterdam changes throughout the year. Altair natively supports dates and discretization of dates when we set the type to temporal (shorthand `T`). For example, in the following plot, we compute the total precipitation for each `month`. To discretize the data into months, we can use a `month` binning (see [TimeUnit](https://altair-viz.github.io/user_guide/transform/timeunit.html#user-guide-timeunit-transform) for more information about this and other `timeUnit` binnings):

In [4]:
alt.Chart(df).mark_line().encode(
    x='month(date):T',
    y='average(precipitation)'
)

alt.Chart(...)

This chart shows that in Rotterdam the precipitation in the autumn and winter is, on average, higher than in spring and summer, but rain can be expected in any month (an unsurprising observation to those who live there!). By changing the mapping of encoding channels to data features, you can begin to explore the relationships within the data.

When looking at precipitation and temperature, we might want to aggregate by year _and_ month (`yearmonth`) rather than just month. This allows us to see seasonal trends, with daily variation smoothed out. We might also wish to see the maximum and minimum temperature in each month:

In [5]:
alt.Chart(df).mark_line().encode(
    x='yearmonth(date):T',
    y='max(temp_max)',
)

alt.Chart(...)

In this chart, it looks as though the maximum temperature fluctuates somewhat over years. To look closer into this, let’s instead look at the mean of the maximum daily temperatures for each year:

In [6]:
alt.Chart(df).mark_line().encode(
    x='year(date):T',
    y='mean(temp_max)',
)

alt.Chart(...)

This can be a little clearer if we use a bar plot and mark the year as an “ordinal” (ordered category) type. For aesthetic reasons, let’s make the bar chart horizontal by assigning the ordinal value to the y-axis:

In [7]:
alt.Chart(df).mark_bar().encode(
    x='mean(temp_max)',
    y='year(date):O'
)

alt.Chart(...)

The chart indicates that the annual average of the daily high temperatures fluctuates over the years, something that you can confirm for minimum daily temperatures as well.

You might also wonder how the daily temperature range changes throughout the year. For this, we have to add a computation to derive a new field, which can be done by adding a `calculate` transform:

In [8]:
alt.Chart(df).mark_bar().encode(
    x='mean(temp_range):Q',
    y='year(date):O'
).transform_calculate(
    temp_range="datum.temp_max - datum.temp_min"
)

alt.Chart(...)

Note that this calculation doesn’t actually do any data manipulation in Python, but rather encodes and stores the operations within the plot specification, where they will be calculated by the renderer.

Of course, the same calculation could be done by using pandas manipulations to explicitly add a column to the dataframe; the disadvantage there is that the derived values would have to be stored in the plot specification rather than computed on-demand in the browser.

Next we will explore the `weather` field, which encodes a categorical variable describing the weather on a given day. We might wish to know how different kinds of weather (e.g. sunny days or rainy days) are distributed throughout the year. To answer this, we can discretize the date by month and then count the number of records on the y-axis. We then break down the bars by the weather type by mapping this column to a color channel. When a bar chart has a field mapped to color, Altair will automatically stack the bars atop each other:

In [9]:
alt.Chart(df).mark_bar().encode(
    x='month(date):N',
    y='count()',
    color='weather',
)

alt.Chart(...)

The default color palette’s semantics might not match our expectation. We can tune the chart by providing a color scale range that maps the values from the weather field to aesthetic and meaningful colors, using standard hex color codes:

In [10]:
scale = alt.Scale(domain=['sun', 'fog', 'ice', 'rain', 'snow'],
                  range=['#e7ba52', '#c7c7c7', '#aec7e8', '#1f77b4', '#9467bd'])

This scale can be passed to the color encoding to be applied to the plot style. In addition, we can customize the titles for the axis and legend to make the meaning of the plot more clear:

In [11]:
alt.Chart(df).mark_bar().encode(
    x=alt.X('month(date):N').title('Month of the year'),
    y='count()',
    color=alt.Color('weather').legend(title='Weather type').scale(scale)
)

alt.Chart(...)

Combining the above ideas lets us create any number of flexible visualizations of this dataset. For example, here is a plot that uses the customizations we have developed above to explore the relationship between weather, precipitation, maximum temperature, configured to use a larger canvas and to allow interactive panning and zooming with the mouse:

In [12]:
alt.Chart(df).mark_point(tooltip=True).encode(
    alt.X('monthdate(date):T').title('Date'),
    alt.Y('temp_max:Q')
        .title('Maximum Daily Temperature (C)')
        .scale(domain=[-5, 40]),
    alt.Size('precipitation:Q').scale(range=[5, 200]),
    color=alt.Color('weather').legend(title='Weather type').scale(scale)
).properties(
    width='container'
).interactive()

alt.Chart(...)

This gives us even more insight into the weather patterns in Rotterdam: rainy and foggy days tend to be cooler, while warmer days tend to be dry and sunny, with a wider spread between low and high temperature.

You can take this even further using Altair’s building blocks for multi-panel charts and interactions. For example, we might construct a histogram of days by weather type:

In [13]:
alt.Chart(df).mark_bar().encode(
    x='count()',
    y='weather:N',
    color=alt.Color('weather:N').scale(scale),
)

alt.Chart(...)

And now we can vertically concatenate this histogram to the points plot above, and add a brush selection tool such that the histogram reflects the content of the selection (for more information on selections, see [Interactive Charts](https://altair-viz.github.io/user_guide/interactions.html)):

In [14]:
# we create two selections:
# - a brush that is active on the top panel
# - a multi-click that is active on the bottom panel
brush = alt.selection_interval(encodings=['x'])
click = alt.selection_point(encodings=['color'])

# top panel is scatter plot of temperature vs time
points = alt.Chart().mark_point().encode(
    alt.X('monthdate(date):T').title('Date'),
    alt.Y('temp_max:Q')
        .title('Maximum Daily Temperature (C)')
        .scale(domain=[-5, 40]),
    alt.Size('precipitation:Q').scale(range=[5, 200]),
    color=alt.condition(brush, alt.Color('weather:N').scale(scale), alt.value('lightgray')),
).properties(
    width=550,
    height=300
).add_params(
    brush
).transform_filter(
    click
)

# bottom panel is a bar chart of weather type
bars = alt.Chart().mark_bar().encode(
    x='count()',
    y='weather:N',
    color=alt.condition(click, alt.Color('weather:N').scale(scale), alt.value('lightgray')),
).transform_filter(
    brush
).properties(
    width=550,
).add_params(
    click
)

alt.vconcat(
    points,
    bars,
    data=df,
    title="Rotterdam Weather: 2017-2022"
)

alt.VConcatChart(...)

This chart, containing concatenations, data transformations, selections, and customized axes labels and data scales, shows the power of the grammar behind Altair: you can create a complex chart from a small number of building blocks.

This is the end of this tutorial where you have seen various ways to bin and aggregate data, derive new fields, and customize your charts. You can find more visualizations in the [Example Gallery](https://altair-viz.github.io/gallery/). If you want to further customize your charts, you can refer to Altair’s [API Reference](https://altair-viz.github.io/user_guide/api.html).